<a href="https://colab.research.google.com/github/arssite/GENAi-Assessment/blob/main/ObjectDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm transformers torch gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.

In [4]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont


# Use a pipeline as a high-level helper
from transformers import pipeline

# model_path = ("../Models/models--facebook--detr-resnet-50/snapshots"
#               "/1d5f47bd3bdd2c4bbfa585418ffe6da5028b4c0b")

object_detector = pipeline("object-detection",
                model="facebook/detr-resnet-50")

# object_detector = pipeline("object-detection",
#                 model=model_path)


def draw_bounding_boxes(image, detections, font_path=None, font_size=20):
    """
    Draws bounding boxes on the given image based on the detections.
    :param image: PIL.Image object
    :param detections: List of detection results, where each result is a dictionary containing
                       'score', 'label', and 'box' keys. 'box' itself is a dictionary with 'xmin',
                       'ymin', 'xmax', 'ymax'.
    :param font_path: Path to the TrueType font file to use for text.
    :param font_size: Size of the font to use for text.
    :return: PIL.Image object with bounding boxes drawn.
    """
    # Make a copy of the image to draw on
    draw_image = image.copy()
    draw = ImageDraw.Draw(draw_image)

    # Load custom font or default font if path not provided
    if font_path:
        font = ImageFont.truetype(font_path, font_size)
    else:
        # When font_path is not provided, load default font but it's size is fixed
        font = ImageFont.load_default()
        # Increase font size workaround by using a TTF font file, if needed, can download and specify the path

    for detection in detections:
        box = detection['box']
        xmin = box['xmin']
        ymin = box['ymin']
        xmax = box['xmax']
        ymax = box['ymax']

        # Draw the bounding box
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline="red", width=3)

        # Optionally, you can also draw the label and score
        label = detection['label']
        score = detection['score']
        text = f"{label} {score:.2f}"

        # Draw text with background rectangle for visibility
        if font_path:  # Use the custom font with increased size
            text_size = draw.textbbox((xmin, ymin), text, font=font)
        else:
            # Calculate text size using the default font
            text_size = draw.textbbox((xmin, ymin), text)

        draw.rectangle([(text_size[0], text_size[1]), (text_size[2], text_size[3])], fill="red")
        draw.text((xmin, ymin), text, fill="white", font=font)

    return draw_image


def detect_object(image):
    raw_image = image
    lst=[]
    output = object_detector(raw_image)
    for i in output:
      lst.append(i['label'])
    processed_image = draw_bounding_boxes(raw_image, output)
    return processed_image,lst

demo = gr.Interface(fn=detect_object,
                    inputs=[gr.Image(label="Select Image",type="pil")],
                    outputs=[gr.Image(label="Processed Image", type="pil"),gr.Textbox(label="Objcts", lines=3),],
                    title="@GenAILearniverse Project 6: Object Detector",
                    description="THIS APPLICATION WILL BE USED TO DETECT OBJECTS INSIDE THE PROVIDED INPUT IMAGE.")
demo.launch()

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argum

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7a5b2d4eafce0df508.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
!gradio deploy

Upload 2 LFS files:   0% 0/2 [00:00<?, ?it/s]
mnist_test.csv:   0% 0.00/18.3M [00:00<?, ?B/s]

mnist_train_small.csv:   0% 0.00/36.5M [00:00<?, ?B/s]
mnist_test.csv:  15% 2.74M/18.3M [00:00<00:00, 19.0MB/s]

mnist_train_small.csv:   7% 2.72M/36.5M [00:00<00:01, 18.6MB/s]

mnist_train_small.csv:  13% 4.59M/36.5M [00:00<00:02, 14.8MB/s]
mnist_test.csv:  25% 4.65M/18.3M [00:00<00:00, 14.6MB/s]

mnist_train_small.csv:  21% 7.77M/36.5M [00:00<00:01, 18.3MB/s]
mnist_test.csv:  41% 7.52M/18.3M [00:00<00:00, 16.4MB/s]
mnist_test.csv:  52% 9.45M/18.3M [00:00<00:00, 16.6MB/s]
mnist_test.csv:  63% 11.6M/18.3M [00:00<00:00, 16.2MB/s]

mnist_train_small.csv:  33% 11.9M/36.5M [00:00<00:01, 15.7MB/s]
mnist_test.csv:  82% 15.0M/18.3M [00:00<00:00, 17.9MB/s]
mnist_test.csv:  92% 16.7M/18.3M [00:01<00:00, 10.5MB/s]

mnist_train_small.csv:  44% 16.0M/36.5M [00:01<00:01, 10.7MB/s]

mnist_train_small.csv:  82% 29.9M/36.5M [00:01<00:00, 28.0MB/s]

mnist_test.csv: 100% 18.3M/18.3M [00:02<00:00, 8.04MB/s]
mni